## Expanding the CIP netCDFfiles and adding data from nav files



In [1]:
import xarray as xr # read netcdf-files
import numpy as np
import warnings
import pandas as pd
import glob # allows for wildcards in filemanagement
import os #get a list of all directories/files
import re #regex

import functions

In [58]:
# sample rate to process (current possibilities: 1,5,12 sek)
sample_rate = 1

In [59]:
# -- Paths to datafiles
# Local disk path to nav data:
nav_main_path = '../2022-islas/' # directory with flight data
nav_file_struct_tdyn = '/*_TDYN_*.nc' # structure of nav TDYN file names
nav_file_struct_nav = '/*_NAV_*.nc' # structure of nav NAV file names
drop_flights = ['as220005','as220006'] # flights to drop, (if not all are to be analysed 5 and 6 is in france)
    
# Local disk path to SODA processed CIP files
cip_main_path = f'../Results_2022-islas/{sample_rate}sAveraging/'
cip_file_struct = '/*CIP.nc'

# Save file path
save_path = '../Results_2022-islas/Processed/CIP_processed/'

In [60]:
# -- Get foldernames that are flights (valid in both nav_main_path and cip_main_part)    
# regex for only using folders that are flights
patt = re.compile(r"as2200\d{2}") # flights have the pattern as2200 + 2 digits
     
flights = [
    f for f in os.listdir(nav_main_path) 
    if os.path.isdir(os.path.join(nav_main_path, f)) and patt.fullmatch(f)
    ]
     
# remove flights to drop using listcomprehension
flights = [i for i in flights if i not in drop_flights] # list of folders/flights

# safireid-islasid dictionary
safire_to_islas = {
    'as220007':'IS22-02',
    'as220008':['IS22-03','IS22-04'],
    'as220009':'IS22-05',
    'as220010':'IS22-06',
    'as220011':'IS22-07',
    'as220012':'IS22-08',
    'as220013':'IS22-09',
    'as220014':'IS22-10',
    'as220015':'IS22-11'
    }

In [68]:
# Process all flights in the folders
for flight in flights:
    # ---- Get CDP and NAV data from flight
    if flight == 'as220008':
        nav_file = glob.glob(nav_main_path + flight + nav_file_struct_tdyn) 
        # flight with safireid as 220008 has two islasids and must be addressed separately:
        for islasid in safire_to_islas['as220008']:
            cip_file = glob.glob(cip_main_path + flight + f'/{islasid}' + cip_file_struct) 
            print(f'Reading: {cip_file[0]} and {nav_file[0]}')
            cip_updated_xds = standardize_cip_netcdf(cip_file[0], nav_file[0], islasid)

            # save to new netCDF
            savefile = save_path + f'CIP_update_{sample_rate}s_{islasid}.nc'
            nc_save_with_check(savefile, cip_updated_xds)

            
    else:
        nav_file = glob.glob(nav_main_path + flight + nav_file_struct_tdyn) 
        cip_file = glob.glob(cip_main_path + flight + cip_file_struct) # returns a list, must access with file[0]
        #cip_xds = xr.open_dataset(cip_file) # returns an xarray dataset
        print(f'Reading: {cip_file[0]} and {nav_file[0]}')
        islasid = safire_to_islas[flight]
        cip_updated_xds = standardize_cip_netcdf(cip_file[0], nav_file[0], islasid)

        #Save to new netCDF
        savefile = save_path + f'CIP_update_{sample_rate}s_{islasid}.nc'      
        nc_save_with_check(savefile, cip_updated_xds)

Reading: ../Results_2022-islas/1sAveraging/as220014/04032022_071513_CIP.nc and ../2022-islas/as220014/ISLAS_SAFIRE-ATR42_CORE_TDYN_1HZ_20220403_as220014_L1_V1.nc
Saving to ../Results_2022-islas/Processed/CIP_processed/CIP_update_1s_IS22-10.nc
Reading: ../Results_2022-islas/1sAveraging/as220009/03262022_080110_CIP.nc and ../2022-islas/as220009/ISLAS_SAFIRE-ATR42_CORE_TDYN_1HZ_20220326_as220009_L1_V1.nc
Saving to ../Results_2022-islas/Processed/CIP_processed/CIP_update_1s_IS22-05.nc
Reading: ../Results_2022-islas/1sAveraging/as220010/03262022_145034_CIP.nc and ../2022-islas/as220010/ISLAS_SAFIRE-ATR42_CORE_TDYN_1HZ_20220326_as220010_L1_V1.nc
Saving to ../Results_2022-islas/Processed/CIP_processed/CIP_update_1s_IS22-06.nc
Reading: ../Results_2022-islas/1sAveraging/as220007/03222022_113721_CIP.nc and ../2022-islas/as220007/ISLAS_SAFIRE-ATR42_CORE_TDYN_1HZ_20220322_as220007_L1_V1.nc
Saving to ../Results_2022-islas/Processed/CIP_processed/CIP_update_1s_IS22-02.nc
Reading: ../Results_2022-isl

In [67]:
def nc_save_with_check(savefile ,xds:xr.Dataset)->None:
    """Check if a netCDF file exists. Overwrite existing if user accepts, create new if not existing.

      Parameters
    ----------
    savefile
        Path to netCDF file
    xds
        xarray.DataSet that user wants to write to savefile
    
    """
    # Check if the file exists
    if os.path.exists(savefile):
        overwrite = input(f'The file {savefile} exists. Do you want to overwrite it?(y/n)')
        if overwrite.lower() in ["yes", "y"]:
            print(f'Saving to {savefile}')
            xds.to_netcdf(path=savefile, mode='a')
        else:
            print("Exiting...")
    else:
        xds.to_netcdf(path=savefile, mode='w')
        print(f'Saving to {savefile}')
        
    return

nc_save_with_check(savefile, cip_updated_xds)

Exiting...


In [ ]:
# testing
#cip_updated_xds

NameError: name 'user_input' is not defined

In [32]:
# function to add NAV information to CIP xarray and standardize file to CF/ACDD
# TODO: Update metadata

def standardize_cip_netcdf(cip_nc_file, nav_file, flight):
    from datetime import date
    import xarray as xr
    import numpy as np
    import re
    import functions
    
    cip_xds = xr.open_dataset(cip_nc_file) # returns an xarray dataset
    nav_xds = xr.open_dataset(nav_file) # returns an xarray dataset

    # fix time dimension cip_xds
    cip_xds = cip_xds.rename_vars({'elapsed_time':'time'}) # elapsed time holds the correct time to use, change name to time for simplisity
    cip_xds = cip_xds.set_coords('time') # set as coordinate 
    cip_xds = cip_xds.swap_dims({'Time': 'time'}) # set as main dimension
    cip_xds = functions.floor_to_sec_res(cip_xds,'time') # floor the times to sec for easier joining

    # drop duplicate time steps (in nav)
    index = np.unique(nav_xds.time, return_index = True)[1]
    nav_xds = nav_xds.isel(time=index)
    
    #print(f'nav_min: {nav_xds.time.values.min()}, nav max: {nav_xds.time.values.max()}')

    # -- UPDATE COORDINATES based on NAV file
    datetimes = cip_xds.time.values #base_time + times.astype('timedelta64[s]')                   #transform from seconds from midnight to datetimeobject 
    # cip_min = datetimes.min()
    # cip_max = datetimes.max()
    # print(f'cip_min: {cip_min}, cip_max: {cip_max}')
    
    
    # select the NAV data from these times
    sel_data = nav_xds.sel(time=datetimes, method = "nearest")           # "nearest" due to diffs in decimalseconds

    # Add NAV coordinates to the CIP xarray
    cip_updated_xds = cip_xds.assign_coords(sel_data.coords)
    
    # change name of LWC ( to avoid confusion with cdp later)
    cip_updated_xds = cip_updated_xds.rename({'LWC': 'LWC_cip'})

    # add MET variables from Nav file: Temp, Pres, WS/WD, (humidity parameters also avaliable)
    cip_updated_xds['T'] = sel_data['TEMP1']
    cip_updated_xds['T'].attrs['origin file']=nav_file

    cip_updated_xds['P'] = sel_data['PRES']
    cip_updated_xds['P'].attrs['origin file']=nav_file

    cip_updated_xds['WS'] = sel_data['WS']
    cip_updated_xds['WS'].attrs['origin file']=nav_file

    cip_updated_xds['WD'] = sel_data['WD']
    cip_updated_xds['WD'].attrs['origin file']=nav_file

   
    # change names of lat, lon and alt
    cip_updated_xds = cip_updated_xds.rename({'LATITUDE': 'lat','LONGITUDE': 'lon', 'ALTITUDE':'alt'})

    # Calculate SV for CIP and CDP
    #calculate the sample volume (sample area SA * TAS redused * sample time

    # sample area has one value per bin, and are already given in m2: test['SA'].values
    # sample time is given as seconds in the attribute 'RATE': test.attrs['RATE']
    # TAS should be the same as the one used for the rest of the variables: test['TAS'
    cip_updated_xds['SV_CIP'] = cip_updated_xds['SA']*cip_updated_xds['TAS']*cip_updated_xds.attrs['RATE']
    cip_updated_xds['SV_CIP'].attrs['longname']='Sample volume'
    cip_updated_xds['SV_CIP'].attrs['unit']='m3'
    cip_updated_xds['SV_CIP'].attrs['description']='Sample volume per size bin'
    cip_updated_xds['SV_CIP'].attrs['calculated from']=['SA', 'TAS', 'attrs:RATE']

    #calculate the sample volume (sample area SA * TAS redused * sample time (1 sek))
    # sample area from meta information and given in mm^2 readjust to m by dividing with 10⁶
    #sa = float(meta_df.loc[meta_df['Metadata'] == 'Sample Area (mm^2)', 'Value'].iloc[0]) /(1000*1000)
    #st = meta_df.loc[meta_df['Metadata'] == 'Sample Time (sec)', 'Value'].iloc[0] 
    #islas_cdp_df['SV (m^3)'] = sa * islas_cdp_df['TAS probe reduction (m/s)'] * st

    # -- UPDATE METADATA

    # Global metadata - campaign specifics
    safireid = re.search('as\d+', nav_file)
    cip_updated_xds.attrs['safireid'] = safireid.group()
    cip_updated_xds.attrs['islasid'] = flight

    # Global metadata
    #ACDD Highly recommended
    cip_updated_xds.attrs['title'] = f'CIP dataset from flight {flight} of the ISLAS campaign'
    cip_updated_xds.attrs['summary'] = f'SODA2 Processed CIP data from flight {flight} from the ISLAS 2022 campain. Updated with latitude, longitude, altitude and time coordinates from the flights NAV data'
    cip_updated_xds.attrs['keywords'] = ['Earth Science > Atmosphere > Clouds > Cloud Microphysics > Cloud Liquid Water/Ice',
                                        'Earth Science > Atmosphere > Clouds > Cloud Microphysics > Cloud Droplet Concentration/Size']
    cip_updated_xds.attrs['keywords_vocabulary'] = "GCMD Science Keywords"
    cip_updated_xds.attrs['Conventions'] = ''
    # Filenames used to create this file
    navfilename = nav_file.split('/')[-1]
    cip_updated_xds.attrs['NAV file']=navfilename
    cipfilename = cip_nc_file.split('/')[-1]
    cip_updated_xds.attrs['SODA NC file']=cipfilename

    # Global metadata - ACDD recommended

    # Global metadata - suggested
    cip_updated_xds.attrs['date_modified'] = date.today().strftime("%Y-%m-%d")
    cip_updated_xds.attrs['date_metadata_modified'] = date.today().strftime("%Y-%m-%d")


    cip_xds.close()
    nav_xds.close()
    return cip_updated_xds

In [13]:
# testing not allin files
# Nav files (per safireid)
nav_file_7 = '../2022-islas/as220007/ISLAS_SAFIRE-ATR42_CORE_NAV_1HZ_20220322_as220007_L1_V1.nc'
nav_file_8 = '../2022-islas/as220008/ISLAS_SAFIRE-ATR42_CORE_NAV_1HZ_20220324_as220008_L1_V1.nc'
nav_file_9 = '../2022-islas/as220009/ISLAS_SAFIRE-ATR42_CORE_NAV_1HZ_20220326_as220009_L1_V1.nc'
nav_file_10 = '../2022-islas/as220010/ISLAS_SAFIRE-ATR42_CORE_NAV_1HZ_20220326_as220010_L1_V1.nc'
nav_file_11 = '../2022-islas/as220011/ISLAS_SAFIRE-ATR42_CORE_NAV_1HZ_20220329_as220011_L1_V1.nc'
nav_file_12 = '../2022-islas/as220012/ISLAS_SAFIRE-ATR42_CORE_NAV_1HZ_20220330_as220012_L1_V1.nc'
nav_file_13 = '../2022-islas/as220013/ISLAS_SAFIRE-ATR42_CORE_NAV_1HZ_20220331_as220013_L1_V1.nc'
nav_file_14 = '../2022-islas/as220014/ISLAS_SAFIRE-ATR42_CORE_NAV_1HZ_20220403_as220014_L1_V1.nc'
nav_file_15 = '../2022-islas/as220015/ISLAS_SAFIRE-ATR42_CORE_NAV_1HZ_20220403_as220015_L1_V1.nc'

save_path = '../Results_2022-islas/temp/allin_off/Processed/CIP_update_notallin_'
data_path = '../Results_2022-islas/temp/allin_off/'

cip_updated_xds = standardize_cip_netcdf(data_path + '03222022_113721_CIP.nc',nav_file_7,'IS22-02')
cip_updated_xds.to_netcdf(path=save_path+'IS22-02.nc', mode='w')
cip_updated_xds = standardize_cip_netcdf(data_path + '03242022_080425_CIP.nc',nav_file_8,'IS22-03')
cip_updated_xds.to_netcdf(path=save_path+'IS22-03.nc', mode='w')
cip_updated_xds = standardize_cip_netcdf(data_path + '03242022_130925_CIP.nc',nav_file_8,'IS22-04')
cip_updated_xds.to_netcdf(path=save_path+'IS22-04.nc', mode='w')
cip_updated_xds = standardize_cip_netcdf(data_path + '03262022_080110_CIP.nc',nav_file_9,'IS22-05')
cip_updated_xds.to_netcdf(path=save_path+'IS22-05.nc', mode='w')
cip_updated_xds = standardize_cip_netcdf(data_path + '03262022_145034_CIP.nc',nav_file_10,'IS22-06')
cip_updated_xds.to_netcdf(path=save_path+'IS22-06.nc', mode='w')
cip_updated_xds = standardize_cip_netcdf(data_path + '03292022_081437_CIP.nc',nav_file_11,'IS22-07')
cip_updated_xds.to_netcdf(path=save_path+'IS22-07.nc', mode='w')
cip_updated_xds = standardize_cip_netcdf(data_path + '03302022_134849_CIP.nc',nav_file_12,'IS22-08')
cip_updated_xds.to_netcdf(path=save_path+'IS22-08.nc', mode='w')
cip_updated_xds = standardize_cip_netcdf(data_path + '03312022_093432_CIP.nc',nav_file_13,'IS22-09')
cip_updated_xds.to_netcdf(path=save_path+'IS22-09.nc', mode='w')
cip_updated_xds = standardize_cip_netcdf(data_path + '04032022_071513_CIP.nc',nav_file_14,'IS22-10')
cip_updated_xds.to_netcdf(path=save_path+'IS22-10.nc', mode='w')
cip_updated_xds = standardize_cip_netcdf(data_path + '04032022_123844_CIP.nc',nav_file_15,'IS22-11')
cip_updated_xds.to_netcdf(path=save_path+'IS22-11.nc', mode='w')


In [26]:
# testing including meteorology in datafiles
# Nav files (per safireid)
nav_file_7 = '../2022-islas/as220007/ISLAS_SAFIRE-ATR42_CORE_TDYN_1HZ_20220322_as220007_L1_V1.nc'
nav_file_8 = '../2022-islas/as220008/ISLAS_SAFIRE-ATR42_CORE_TDYN_1HZ_20220324_as220008_L1_V1.nc'
nav_file_9 = '../2022-islas/as220009/ISLAS_SAFIRE-ATR42_CORE_TDYN_1HZ_20220326_as220009_L1_V1.nc'
nav_file_10 = '../2022-islas/as220010/ISLAS_SAFIRE-ATR42_CORE_TDYN_1HZ_20220326_as220010_L1_V1.nc'
nav_file_11 = '../2022-islas/as220011/ISLAS_SAFIRE-ATR42_CORE_TDYN_1HZ_20220329_as220011_L1_V1.nc'
nav_file_12 = '../2022-islas/as220012/ISLAS_SAFIRE-ATR42_CORE_TDYN_1HZ_20220330_as220012_L1_V1.nc'
nav_file_13 = '../2022-islas/as220013/ISLAS_SAFIRE-ATR42_CORE_TDYN_1HZ_20220331_as220013_L1_V1.nc'
nav_file_14 = '../2022-islas/as220014/ISLAS_SAFIRE-ATR42_CORE_TDYN_1HZ_20220403_as220014_L1_V1.nc'
nav_file_15 = '../2022-islas/as220015/ISLAS_SAFIRE-ATR42_CORE_TDYN_1HZ_20220403_as220015_L1_V1.nc'


save_path = '../Results_2022-islas/temp/with_MET/'

cip_updated_xds = standardize_cip_netcdf('../Results_2022-islas/as220007/03222022_113721_CIP.nc',nav_file_7,'IS22-02')
cip_updated_xds.to_netcdf(path=save_path+'IS22-02.nc', mode='w')

cip_updated_xds = standardize_cip_netcdf('../Results_2022-islas/as220008/5sek/IS22-03/03242022_080425_CIP.nc',nav_file_8,'IS22-03')
cip_updated_xds.to_netcdf(path=save_path+'IS22-03.nc', mode='w')

cip_updated_xds = standardize_cip_netcdf('../Results_2022-islas/as220008/5sek/IS22-04/03242022_130925_CIP.nc',nav_file_8,'IS22-04')
cip_updated_xds.to_netcdf(path=save_path+'IS22-04.nc', mode='w')

cip_updated_xds = standardize_cip_netcdf('../Results_2022-islas/as220009/03262022_080110_CIP.nc',nav_file_9,'IS22-05')
cip_updated_xds.to_netcdf(path=save_path+'IS22-05.nc', mode='w')

cip_updated_xds = standardize_cip_netcdf('../Results_2022-islas/as220010/03262022_145034_CIP.nc',nav_file_10,'IS22-06')
cip_updated_xds.to_netcdf(path=save_path+'IS22-06.nc', mode='w')

cip_updated_xds = standardize_cip_netcdf('../Results_2022-islas/as220011/03292022_081437_CIP.nc',nav_file_11,'IS22-07')
cip_updated_xds.to_netcdf(path=save_path+'IS22-07.nc', mode='w')

cip_updated_xds = standardize_cip_netcdf('../Results_2022-islas/as220012/03302022_134849_CIP.nc',nav_file_12,'IS22-08')
cip_updated_xds.to_netcdf(path=save_path+'IS22-08.nc', mode='w')

cip_updated_xds = standardize_cip_netcdf('../Results_2022-islas/as220013/03312022_093432_CIP.nc',nav_file_13,'IS22-09')
cip_updated_xds.to_netcdf(path=save_path+'IS22-09.nc', mode='w')

cip_updated_xds = standardize_cip_netcdf('../Results_2022-islas/as220014/04032022_071513_CIP.nc',nav_file_14,'IS22-10')
cip_updated_xds.to_netcdf(path=save_path+'IS22-10.nc', mode='w')

cip_updated_xds = standardize_cip_netcdf('../Results_2022-islas/as220015/04032022_123844_CIP.nc',nav_file_15,'IS22-11')
cip_updated_xds.to_netcdf(path=save_path+'IS22-11.nc', mode='w')

In [ ]:


# as220008
flight= 'IS22-03'
cip_nc_file_5 = '../Results_2022-islas/as220008/5sek/IS22-03/03242022_080425_CIP.nc'
cip_nc_file_12 = '../Results_2022-islas/12sAveraging/as220008/IS22-03/03242022_080425_CIP.nc'
nav_file = '../2022-islas/as220008/ISLAS_SAFIRE-ATR42_CORE_NAV_1HZ_20220324_as220008_L1_V1.nc'
save_path_5 = '../Results_2022-islas/Processed/CIP_processed/CIP_update_5s_IS22-03.nc'
save_path_12 = '../Results_2022-islas/Processed/CIP_processed/CIP_update_12s_IS22-03.nc'

cip_updated_xds = standardize_cip_netcdf(cip_nc_file_5, nav_file, flight)
cip_updated_xds.to_netcdf(path=save_path_5, mode='w')
cip_updated_xds = standardize_cip_netcdf(cip_nc_file_12, nav_file, flight)
cip_updated_xds.to_netcdf(path=save_path_12, mode='w')


In [27]:
cip_xds = xr.open_dataset(save_path+'IS22-11.nc') # returns an xarray dataset
cip_xds

<xarray.Dataset> Size: 4MB
Dimensions:           (time: 2597, Vector64: 64, Vector40: 40)
Coordinates:
  * time              (time) datetime64[ns] 21kB 2022-04-03T12:38:44.240022 ....
    lat               (time) float32 10kB ...
    lon               (time) float32 10kB ...
    alt               (time) float32 10kB ...
Dimensions without coordinates: Vector64, Vector40
Data variables: (12/37)
    base_time         datetime64[ns] 8B ...
    utc_time          (time) float64 21kB ...
    TAS               (time) float32 10kB ...
    MIDBINS           (Vector64) float32 256B ...
    ACTIVETIME        (time) float32 10kB ...
    SA                (Vector64) float32 256B ...
    ...                ...
    MEAN_ASPECTRATIO  (Vector64, time) float32 665kB ...
    T                 (time) float32 10kB ...
    P                 (time) float32 10kB ...
    WS                (time) float32 10kB ...
    WD                (time) float32 10kB ...
    SV_CIP            (Vector64, time) float32 665kB ...
Attributes: (12/62)
    Source:                  SODA-2 OAP Processing Software
    DateCreated:             Mon Feb  5 11:40:28 2024
    FlightDate:              04/03/2022
    DateProcessed:           Mon Feb  5 11:38:30 2024
    TimeInterval:            12:38:44-16:15:04
    FN:                      /home/ninalar/Documents/MC2/2022-islas/as220015/...
    ...                      ...
    keywords_vocabulary:     GCMD Science Keywords
    Conventions:             
    NAV file:                ISLAS_SAFIRE-ATR42_CORE_TDYN_1HZ_20220403_as2200...
    SODA NC file:            04032022_123844_CIP.nc
    date_modified:           2025-04-04
    date_metadata_modified:  2025-04-04

In [7]:

# as220007
cip_nc_file_12 = '../Results_2022-islas/12sAveraging/as220007/03222022_113721_CIP.nc'
cip_nc_file_5 = '../Results_2022-islas/as220007/03222022_113721_CIP.nc'

nav_file = '../2022-islas/as220007/ISLAS_SAFIRE-ATR42_CORE_NAV_1HZ_20220322_as220007_L1_V1.nc'
flight = 'IS22-02'
save_path_5 = f'../Results_2022-islas/Processed/CIP_processed/CIP_update_5s_{flight}.nc'
save_path_12 = f'../Results_2022-islas/Processed/CIP_processed/CIP_update_12s_{flight}.nc'

cip_updated_xds = standardize_cip_netcdf(cip_nc_file_5, nav_file, flight)
cip_updated_xds.to_netcdf(path=save_path_5, mode='w')
cip_updated_xds = standardize_cip_netcdf(cip_nc_file_12, nav_file, flight)
cip_updated_xds.to_netcdf(path=save_path_12, mode='w')


In [8]:
flight='IS22-04'
cip_nc_file_12 = '../Results_2022-islas/12sAveraging/as220008/IS22-04/03242022_130925_CIP.nc'
cip_nc_file_5 = '../Results_2022-islas/as220008/5sek/IS22-04/03242022_130925_CIP.nc'
nav_file = '../2022-islas/as220008/ISLAS_SAFIRE-ATR42_CORE_NAV_1HZ_20220324_as220008_L1_V1.nc'
save_path_5 = f'../Results_2022-islas/Processed/CIP_processed/CIP_update_5s_{flight}.nc'
save_path_12 = f'../Results_2022-islas/Processed/CIP_processed/CIP_update_12s_{flight}.nc'

cip_updated_xds = standardize_cip_netcdf(cip_nc_file_5, nav_file, flight)
cip_updated_xds.to_netcdf(path=save_path_5, mode='w')
cip_updated_xds = standardize_cip_netcdf(cip_nc_file_12, nav_file, flight)
cip_updated_xds.to_netcdf(path=save_path_12, mode='w')

In [9]:
# as220009
cip_nc_file_12 = '../Results_2022-islas/12sAveraging/as220009/03262022_080110_CIP.nc'
cip_nc_file_5 = '../Results_2022-islas/as220009/03262022_080110_CIP.nc'
nav_file = '../2022-islas/as220009/ISLAS_SAFIRE-ATR42_CORE_NAV_1HZ_20220326_as220009_L1_V1.nc'
flight='IS22-05'
save_path_5 = f'../Results_2022-islas/Processed/CIP_processed/CIP_update_5s_{flight}.nc'
save_path_12 = f'../Results_2022-islas/Processed/CIP_processed/CIP_update_12s_{flight}.nc'

cip_updated_xds = standardize_cip_netcdf(cip_nc_file_5, nav_file, flight)
cip_updated_xds.to_netcdf(path=save_path_5, mode='w')
cip_updated_xds = standardize_cip_netcdf(cip_nc_file_12, nav_file, flight)
cip_updated_xds.to_netcdf(path=save_path_12, mode='w')

In [10]:
# as220010
cip_nc_file_12 = '../Results_2022-islas/12sAveraging/as220010/03262022_145034_CIP.nc'
cip_nc_file_5 = '../Results_2022-islas/as220010/03262022_145034_CIP.nc'
nav_file = '../2022-islas/as220010/ISLAS_SAFIRE-ATR42_CORE_NAV_1HZ_20220326_as220010_L1_V1.nc'
flight='IS22-06'
save_path_5 = f'../Results_2022-islas/Processed/CIP_processed/CIP_update_5s_{flight}.nc'
save_path_12 = f'../Results_2022-islas/Processed/CIP_processed/CIP_update_12s_{flight}.nc'

cip_updated_xds = standardize_cip_netcdf(cip_nc_file_5, nav_file, flight)
cip_updated_xds.to_netcdf(path=save_path_5, mode='w')
cip_updated_xds = standardize_cip_netcdf(cip_nc_file_12, nav_file, flight)
cip_updated_xds.to_netcdf(path=save_path_12, mode='w')

In [11]:
# as220011
cip_nc_file_12 = '../Results_2022-islas/12sAveraging/as220011/03292022_081437_CIP.nc'
cip_nc_file_5 = '../Results_2022-islas/as220011/03292022_081437_CIP.nc'
nav_file = '../2022-islas/as220011/ISLAS_SAFIRE-ATR42_CORE_NAV_1HZ_20220329_as220011_L1_V1.nc'
flight='IS22-07'
save_path_5 = f'../Results_2022-islas/Processed/CIP_processed/CIP_update_5s_{flight}.nc'
save_path_12 = f'../Results_2022-islas/Processed/CIP_processed/CIP_update_12s_{flight}.nc'

cip_updated_xds = standardize_cip_netcdf(cip_nc_file_5, nav_file, flight)
cip_updated_xds.to_netcdf(path=save_path_5, mode='w')
cip_updated_xds = standardize_cip_netcdf(cip_nc_file_12, nav_file, flight)
cip_updated_xds.to_netcdf(path=save_path_12, mode='w')

In [12]:
# as220012
cip_nc_file_12 = '../Results_2022-islas/12sAveraging/as220012/03302022_134849_CIP.nc'
cip_nc_file_5 = '../Results_2022-islas/as220012/03302022_134849_CIP.nc'
nav_file = '../2022-islas/as220012/ISLAS_SAFIRE-ATR42_CORE_NAV_1HZ_20220330_as220012_L1_V1.nc'
flight='IS22-08'
save_path_5 = f'../Results_2022-islas/Processed/CIP_processed/CIP_update_5s_{flight}.nc'
save_path_12 = f'../Results_2022-islas/Processed/CIP_processed/CIP_update_12s_{flight}.nc'

cip_updated_xds = standardize_cip_netcdf(cip_nc_file_5, nav_file, flight)
cip_updated_xds.to_netcdf(path=save_path_5, mode='w')
cip_updated_xds = standardize_cip_netcdf(cip_nc_file_12, nav_file, flight)
cip_updated_xds.to_netcdf(path=save_path_12, mode='w')

In [13]:
# as220013
cip_nc_file_12 = '../Results_2022-islas/12sAveraging/as220013/03312022_093432_CIP.nc'
cip_nc_file_5 = '../Results_2022-islas/as220013/03312022_093432_CIP.nc'
nav_file = '../2022-islas/as220013/ISLAS_SAFIRE-ATR42_CORE_NAV_1HZ_20220331_as220013_L1_V1.nc'
flight='IS22-09'
save_path_5 = f'../Results_2022-islas/Processed/CIP_processed/CIP_update_5s_{flight}.nc'
save_path_12 = f'../Results_2022-islas/Processed/CIP_processed/CIP_update_12s_{flight}.nc'


cip_updated_xds = standardize_cip_netcdf(cip_nc_file_5, nav_file, flight)
cip_updated_xds.to_netcdf(path=save_path_5, mode='w')
cip_updated_xds = standardize_cip_netcdf(cip_nc_file_12, nav_file, flight)
cip_updated_xds.to_netcdf(path=save_path_12, mode='w')

In [14]:
# as220014
cip_nc_file_12 = '../Results_2022-islas/12sAveraging/as220014/04032022_071513_CIP.nc'
cip_nc_file_5 = '../Results_2022-islas/as220014/04032022_071513_CIP.nc'
nav_file = '../2022-islas/as220014/ISLAS_SAFIRE-ATR42_CORE_NAV_1HZ_20220403_as220014_L1_V1.nc'
flight='IS22-10'
save_path_5 = f'../Results_2022-islas/Processed/CIP_processed/CIP_update_5s_{flight}.nc'
save_path_12 = f'../Results_2022-islas/Processed/CIP_processed/CIP_update_12s_{flight}.nc'

cip_updated_xds = standardize_cip_netcdf(cip_nc_file_5, nav_file, flight)
cip_updated_xds.to_netcdf(path=save_path_5, mode='w')
cip_updated_xds = standardize_cip_netcdf(cip_nc_file_12, nav_file, flight)
cip_updated_xds.to_netcdf(path=save_path_12, mode='w')


In [15]:
# as220015
cip_nc_file_12 = '../Results_2022-islas/12sAveraging/as220015/04032022_123844_CIP.nc'
cip_nc_file_5 = '../Results_2022-islas/as220015/04032022_123844_CIP.nc'
nav_file = '../2022-islas/as220015/ISLAS_SAFIRE-ATR42_CORE_NAV_1HZ_20220403_as220015_L1_V1.nc'
flight='IS22-11'
save_path_5 = f'../Results_2022-islas/Processed/CIP_processed/CIP_update_5s_{flight}.nc'
save_path_12 = f'../Results_2022-islas/Processed/CIP_processed/CIP_update_12s_{flight}.nc'

cip_updated_xds = standardize_cip_netcdf(cip_nc_file_5, nav_file, flight)
cip_updated_xds.to_netcdf(path=save_path_5, mode='w')
cip_updated_xds = standardize_cip_netcdf(cip_nc_file_12, nav_file, flight)
cip_updated_xds.to_netcdf(path=save_path_12, mode='w')